<a href="https://colab.research.google.com/github/jana-nf/DataOps_Skrub_ML_AgentsAI/blob/main/DataOps_Skrub_ML_AgentsAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DataOps: Integração de Agentes de IA e ML Clássico com Skrub, governança e auditabilidade Git

### Configuração Inicial e Instalação
Como o Google Colab não vem com o skrub instalado por padrão, precisamos instalá-lo primeiro.

In [1]:
!pip install skrub scikit-learn pandas

import pandas as pd
import numpy as np
from skrub import TableVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 498.3/498.3 kB 30.9 MB/s eta 0:00:00


### Configuração, Montagem do Drive e Instalação
Esta etapa conecta o Colab ao seu Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Carregamento do Dataset do Drive e Limpeza Inicial

In [3]:
file_path = '/content/drive/MyDrive/Colab Notebooks/DataOps_Skrub_ML_AgentsAI/WA_Fn-UseC_-Telco-Customer-Churn.csv'

try:
    df = pd.read_csv(file_path)
    print("Dataset Telco Customer Churn carregado com sucesso do Google Drive!")
except FileNotFoundError:
    print(f"ERRO: Arquivo não encontrado no caminho: {file_path}")
    print("Verifique se o nome do arquivo e o caminho no seu Drive estão corretos.")
    exit()

# 1. Pré-limpeza crucial: O campo 'TotalCharges'
# Converte a coluna 'TotalCharges' (que está como string com espaços) para float.
df['TotalCharges'] = df['TotalCharges'].replace(' ', np.nan).astype(float)

# 2. Separar Features (X) e Target (y)
X = df.drop(columns=['Churn', 'customerID'])
y = df['Churn'].map({'Yes': 1, 'No': 0})

# 3. Divisão Treino/Teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("\n--- Informações do Dataset ---")
print(f"Shape de Treino: {X_train.shape}")

Dataset Telco Customer Churn carregado com sucesso do Google Drive!

--- Informações do Dataset ---
Shape de Treino: (5634, 19)


### Aplicação do Skrub (TableVectorizer) no Pipeline
Esta etapa encapsula o pré-processamento (Skrub) e o modelo de ML Clássico (Regressão Logística) no pipeline, demonstrando Clean Code e Integração.

In [7]:
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from skrub import TableVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer

numerical_features = X_train.select_dtypes(include=np.number).columns.tolist()

# Criar um transformador para imputar *apenas* as colunas numéricas
# O resto (as strings) será tratado pelo TableVectorizer.
numerical_transformer = SimpleImputer(strategy='median')

# O ColumnTransformer permite aplicar transformações seletivas.
preprocessor = ColumnTransformer(
    transformers=[
        ('num_imputer', numerical_transformer, numerical_features),
        ('passthrough', 'passthrough', X_train.columns.difference(numerical_features))
    ],
    remainder='drop',
    verbose_feature_names_out=False
)

# Definir o Pipeline Final
pipeline_ml_classico = make_pipeline(
    preprocessor,
    TableVectorizer(),
    LogisticRegression(max_iter=1000, random_state=42)
)

# Treinar o Pipeline
print("\n--- Treinando o Pipeline Skrub + Regressão Logística (Corrigido) ---")
pipeline_ml_classico.fit(X_train, y_train)
print("Treinamento concluído com sucesso!")

# Avaliar o Modelo Clássico (ML Clássico)
y_pred = pipeline_ml_classico.predict(X_test)

print("\n--- Avaliação do ML Clássico (Base para o Agente de IA) ---")
print(classification_report(y_test, y_pred, target_names=['No Churn', 'Churn']))


--- Treinando o Pipeline Skrub + Regressão Logística (Corrigido) ---


/usr/local/lib/python3.12/dist-packages/skrub/_check_input.py:175: UserWarning: Only pandas and polars DataFrames are supported, but input is a Numpy array. Please convert Numpy arrays to DataFrames before passing them to skrub transformers. Converting to pandas DataFrame with columns ['0', '1', …].
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/skrub/_clean_null_strings.py:39: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  return col.replace(r"^\s*$", "", regex=True)


Treinamento concluído com sucesso!

--- Avaliação do ML Clássico (Base para o Agente de IA) ---
              precision    recall  f1-score   support

    No Churn       0.85      0.89      0.87      1035
       Churn       0.66      0.56      0.60       374

    accuracy                           0.80      1409
   macro avg       0.75      0.73      0.74      1409
weighted avg       0.80      0.80      0.80      1409



/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.12/dist-packages/skrub/_check_input.py:175: UserWarning: Only pandas and polars DataFrames are supported, but input is a Numpy array. Please convert Numpy arrays to DataFrames before passing them to skrub transformers. Converting to pandas DataFrame with columns ['0', '1', …].
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/skrub/_clean_null_strings.py:39: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future ver

### Geração de Valores SHAP (Interpretabilidade)
Precisamos da biblioteca SHAP. Como estamos usando um Pipeline com TableVectorizer no início, usaremos um Explainer que pode lidar com transformações.

### Instalação

In [9]:
!pip install shap
import shap
import json
import hashlib
import time

### Código SHAP

In [11]:
# Acessar o modelo treinado (LogisticRegression)
model = pipeline_ml_classico.named_steps['logisticregression']

# Acessar o TableVectorizer treinado
vectorizer = pipeline_ml_classico.named_steps['tablevectorizer']

# Acessar o Preprocessor (ColumnTransformer + SimpleImputer) treinado
preprocessor = pipeline_ml_classico.named_steps['columntransformer']

# Transformar os dados de teste usando TODOS os passos de pré-processamento
# Aplicar o ColumnTransformer/Imputer
X_test_imputed = preprocessor.transform(X_test)

# Aplicar o TableVectorizer aos dados imputados.
# O Skrub agora recebe a matriz que ele esperava ter na etapa de fit!
X_test_clean = vectorizer.transform(X_test_imputed)
feature_names_clean = vectorizer.get_feature_names_out()

# Criar o SHAP Explainer
# Usaremos o Explainer para modelos lineares
explainer = shap.Explainer(model, X_test_clean)
shap_values = explainer(X_test_clean)

# Selecionar um cliente de ALTO RISCO (Exemplo para o Agente de IA)
index_risco = np.where(y_test == 1)[0][0]
cliente_risco_X = X_test.iloc[index_risco]

# Para prever a probabilidade do cliente individual, ele deve passar pelo pipeline COMPLETO
probabilidade_churn = pipeline_ml_classico.predict_proba(cliente_risco_X.to_frame().T)[:, 1][0]
shap_local = shap_values[index_risco]

print("\n--- Resultado SHAP (Para o Raciocínio do Agente) ---")
print(f"Probabilidade de Churn do Cliente Selecionado: {probabilidade_churn:.2f}")

# Mapear os valores SHAP para um formato legível pelo Agente (JSON)
shap_df = pd.DataFrame({
    'Feature': feature_names_clean,
    'SHAP_Value': shap_local.values
}).sort_values(by='SHAP_Value', ascending=False)

# Fatores que mais contribuíram POSITIVAMENTE para o Churn (Top 3)
top_churn_factors = shap_df.head(3).to_dict(orient='records')
print(f"Top 3 Fatores de Risco (SHAP): {top_churn_factors}")

/usr/local/lib/python3.12/dist-packages/skrub/_check_input.py:175: UserWarning: Only pandas and polars DataFrames are supported, but input is a Numpy array. Please convert Numpy arrays to DataFrames before passing them to skrub transformers. Converting to pandas DataFrame with columns ['0', '1', …].
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/skrub/_clean_null_strings.py:39: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  return col.replace(r"^\s*$", "", regex=True)
/usr/local/lib/python3.12/dist-packages/skrub/_check_input.py:175: UserWarning: Only pandas and polars DataFrames are supported, but input is a Numpy array. Please convert Numpy arrays to DataFrames before passing them to skrub transformers. Converting to pandas DataFrame with columns 


--- Resultado SHAP (Para o Raciocínio do Agente) ---
Probabilidade de Churn do Cliente Selecionado: 0.40
Top 3 Fatores de Risco (SHAP): [{'Feature': '1', 'SHAP_Value': 0.7309180494810087}, {'Feature': '4_Month-to-month', 'SHAP_Value': 0.2857466390001965}, {'Feature': '11_Yes', 'SHAP_Value': 0.16723687034076076}]


### Simulação do Agente de IA e Log de Auditabilidade
Roteamento Híbrido (FinOps), o Raciocínio (LLM) e a Auditabilidade (Git).

In [12]:
GIT_HASH = hashlib.sha256(str(time.time()).encode()).hexdigest()[:8]
LLM_VERSION = "GPT-4o (Simulado)"
FINOPS_THRESHOLD = 0.70 # Acima de 70% de risco, aciona o Agente caro.

# Lógica do Roteamento Híbrido (FinOps)
if probabilidade_churn >= FINOPS_THRESHOLD:
    print(f"\n--- Roteamento Híbrido ATIVADO: Risco ({probabilidade_churn:.2f}) > Limiar ({FINOPS_THRESHOLD:.2f}) ---")

    # Entrada de Dados Estruturados para o Agente (SHAP + Previsão)
    contexto_para_agente = {
        "Probabilidade_Churn": f"{probabilidade_churn:.2f}",
        "Fatores_Risco": top_churn_factors
    }

    # Simulação do Raciocínio (LLM/Prompt Engineering)
    # O LLM consumiria 'contexto_para_agente' e geraria esta Ação.
    acao_sugerida = f"O Cliente [ID: {cliente_risco_X.name}] tem um risco crítico de {probabilidade_churn:.0%}. A principal causa é o '{contexto_para_agente['Fatores_Risco'][0]['Feature']}'. Ação: Enviar OFERTA VIP + Ligação de Suporte de Nível 3."

    # Log de Auditabilidade e FinOps
    log_auditoria = {
        "timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),
        "cliente_id": cliente_risco_X.name,
        "previsao_ml_classico": probabilidade_churn,
        "decisao_agente": acao_sugerida,
        "modelo_ml_hash": GIT_HASH,
        "llm_usado": LLM_VERSION,
        "fatores_interpretacao_shap": top_churn_factors
    }

    print(f"Decisão do Agente: {acao_sugerida}")
    print("\n--- Log de Auditoria Gerado (Json) ---")
    print(json.dumps(log_auditoria, indent=4))

else:
    # Cenário de Otimização FinOps: LLM não é ativado
    print(f"\n--- Otimização FinOps: Risco ({probabilidade_churn:.2f}) Abaixo do Limiar ---")
    print("Ação: Nenhuma ação dispendiosa com LLM foi tomada.")


--- Otimização FinOps: Risco (0.40) Abaixo do Limiar ---
Ação: Nenhuma ação dispendiosa com LLM foi tomada.


### Outra SIMULAÇÃO

### Setup e Instalação do MLflow

In [13]:
# Instalações adicionais necessárias
!pip install mlflow
import mlflow
import json
import time
import os

# Configurar o MLflow para rastrear localmente (pode ser substituído por um servidor remoto)
mlflow.set_tracking_uri("sqlite:///mlruns.db")
mlflow.set_experiment("churn_hybrid_agent_project")

# Simulação de variáveis de governança (Agora gerenciadas pelo MLflow/Sistema)
FINOPS_THRESHOLD = 0.70
LLM_PROVIDER = "Gemini API (Simulado)"

# Função para simular a chamada da API do LLM
def generate_agent_action(contexto_para_agente, cliente_id):
    """
    Simula uma chamada real a um LLM (como Gemini, GPT, etc.) para raciocínio.
    Em um projeto real, esta função faria uma requisição HTTP para a API.
    """
    # Em um LLM real, você usaria o contexto_para_agente para construir um prompt:
    # prompt = f"O cliente {cliente_id} tem {contexto_para_agente['Probabilidade_Churn']} de churn.
    # Os fatores de risco são: {contexto_para_agente['Fatores_Risco']}. Sugira uma ação tática de retenção."

    prob_churn = float(contexto_para_agente['Probabilidade_Churn'])
    fator_principal = contexto_para_agente['Fatores_Risco'][0]['Feature']

    # Simulação do Raciocínio baseado no fator principal:
    if "MonthlyCharges" in fator_principal or "TotalCharges" in fator_principal:
        sugestao = "Oferta de desconto de 15% para fidelidade e monitoramento do uso."
    elif "tenure" in fator_principal:
        sugestao = "Ligação de Nível 3 para pesquisa de satisfação profunda e proposta de novo contrato."
    else:
        sugestao = "E-mail com oferta de recursos premium gratuitos por um mês."

    acao_sugerida = f"O Cliente [ID: {cliente_id}] tem um risco crítico de {prob_churn:.0%}. A principal causa é o '{fator_principal}'. Ação: {sugestao}"

    return acao_sugerida

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 103.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 107.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 764.2/764.2 kB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 25.2 MB/s eta 0:00:00


2025/12/12 23:56:59 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/12/12 23:56:59 INFO mlflow.store.db.utils: Updating database tables
2025/12/12 23:56:59 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2025/12/12 23:56:59 INFO alembic.runtime.migration: Will assume non-transactional DDL.
2025/12/12 23:56:59 INFO alembic.runtime.migration: Running upgrade  -> 451aebb31d03, add metric step
2025/12/12 23:56:59 INFO alembic.runtime.migration: Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
2025/12/12 23:56:59 INFO alembic.runtime.migration: Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
2025/12/12 23:56:59 INFO alembic.runtime.migration: Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
2025/12/12 23:56:59 INFO alembic.runtime.migration: Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
2025/12/12 23:56:59 INFO alembic.runtime.migration: Running 

### Rastreamento do Treinamento (MLflow)

In [14]:
# Início do Rastreamento MLflow
with mlflow.start_run() as run:

    # Parâmetros de Governança
    mlflow.log_param("finops_threshold", FINOPS_THRESHOLD)
    mlflow.log_param("modelo_ml_algoritmo", "LogisticRegression")

    # Treinar o Pipeline (Skrub + ML Clássico)
    # Assume-se que pipeline_ml_classico, X_train, y_train já estão definidos
    pipeline_ml_classico.fit(X_train, y_train)

    # Avaliar e Logar Métricas (ML Clássico)
    y_pred = pipeline_ml_classico.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    mlflow.log_metric("accuracy", report['accuracy'])
    mlflow.log_metric("f1_score_churn", report['1']['f1-score'])

    # Registrar o Modelo (para rastreabilidade do Skrub + RegLog)
    # MLflow rastreia automaticamente o ambiente e o código que criou o modelo.
    mlflow.sklearn.log_model(
        sk_model=pipeline_ml_classico,
        artifact_path="churn_model_pipeline",
        registered_model_name="Skrub_Hybrid_Churn_Predictor"
    )

    RUN_ID = run.info.run_id
    print(f"\n--- MLflow Treinamento Concluído ---")
    print(f"Modelo registrado com RUN_ID: {RUN_ID}")

/usr/local/lib/python3.12/dist-packages/skrub/_check_input.py:175: UserWarning: Only pandas and polars DataFrames are supported, but input is a Numpy array. Please convert Numpy arrays to DataFrames before passing them to skrub transformers. Converting to pandas DataFrame with columns ['0', '1', …].
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/skrub/_clean_null_strings.py:39: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  return col.replace(r"^\s*$", "", regex=True)
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org


--- MLflow Treinamento Concluído ---
Modelo registrado com RUN_ID: fa0b71b6e1254fd3938ad67bff3b6ad2


Successfully registered model 'Skrub_Hybrid_Churn_Predictor'.
Created version '1' of model 'Skrub_Hybrid_Churn_Predictor'.


### Integração Híbrida e Log de Auditoria Real
Usar a função generate_agent_action (simulação do LLM) e garantir que o log de auditoria capture o ID do MLflow para a rastreabilidade.

In [15]:
# Lógica do Roteamento Híbrido (FinOps)
if probabilidade_churn >= FINOPS_THRESHOLD:

    # Entrada de Dados Estruturados (SHAP + Previsão)
    contexto_para_agente = {
        "Probabilidade_Churn": f"{probabilidade_churn:.2f}",
        "Fatores_Risco": top_churn_factors
    }

    # CHAMADA REAL (Simulada) AO LLM
    acao_sugerida = generate_agent_action(contexto_para_agente, cliente_risco_X.name)

    # Log de Auditabilidade e FinOps
    log_auditoria = {
        "timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),
        "cliente_id": cliente_risco_X.name,
        "previsao_ml_classico": probabilidade_churn,
        "decisao_agente": acao_sugerida,
        "mlflow_run_id": RUN_ID,
        "llm_provider": LLM_PROVIDER,
        "fatores_interpretacao_shap": top_churn_factors
    }

    print(f"\n--- Roteamento Híbrido ATIVADO: (Risco: {probabilidade_churn:.2f}) ---")
    print(f"Decisão do Agente (LLM): {acao_sugerida}")
    print("\n--- Log de Auditoria Final (Para BD de Log) ---")
    print(json.dumps(log_auditoria, indent=4))


    print("\n[Ação Simulado]: Enviado para o sistema de CRM para execução!")

else:
    # Cenário de Otimização FinOps:
    print(f"\n--- Otimização FinOps: Risco ({probabilidade_churn:.2f}) Abaixo do Limiar ---")
    print("Ação: Nenhuma chamada dispendiosa ao LLM foi realizada.")


--- Otimização FinOps: Risco (0.40) Abaixo do Limiar ---
Ação: Nenhuma chamada dispendiosa ao LLM foi realizada.


### Rastreabilidade MLflow: Agora, se a Decisão do Agente estiver errada, você tem o mlflow_run_id para consultar o MLflow e ver a versão exata do código Skrub e do modelo de ML Clássico usado.

### Integração LLM: A função generate_agent_action serve como interface clara, pronta para ser trocada por uma chamada requests.post à API do Gemini ou outro LLM, transformando o contexto_para_agente (os dados SHAP) em um prompt estruturado.